In [12]:
#!/usr/bin/env python
# coding: utf-8
#######################################################################
# How to setup AndroSensor:
# Install AndroSensor
# Ensure when installing, allow app to run in background.
# Open settings and change recording interval to fast.
#
#
#
#######################################################################
import glob
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'
import numpy as np
import scipy as sp
import scipy.signal as sg

px.set_mapbox_access_token(open(".mapbox_token").read())

In [2]:
# IF useing ; sep, then delete the first line.
path = r'D:\OneDrive - Imperial College London\University Storage\Masters project\Raw data'  # use your path
all_files = glob.glob(os.path.join(path, "*.csv")) 

df_from_each_file = [pd.read_csv(f, sep=";") for f in all_files]
concatenated_df = pd.concat(df_from_each_file, ignore_index=True, sort=True)

# Convert datatime to actual datetime
concatenated_df["timestamp"] = pd.to_datetime(concatenated_df["YYYY-MO-DD HH-MI-SS_SSS"],format='%Y-%m-%d %H:%M:%S:%f')
concatenated_df["Speed(km/h)"] = concatenated_df["LOCATION Speed ( Kmh)"]

In [5]:
# calculate abs accleration
concatenated_df["abs_acceleration"] = np.linalg.norm(
    concatenated_df[["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"]].values, axis=1)

# Calculate abs location
concatenated_df["abs_loc"] = np.linalg.norm(concatenated_df[['LOCATION Latitude : ', 'LOCATION Longitude : ']].values,
                                            axis=1)

# Display 3-axis acceleration vs time

In [ ]:
# Display accelration
fig = px.line(concatenated_df,
              x="timestamp",
              y=["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"],
              title='Acceleration'
             )
fig.write_html("plot_acc.html")

fig.show()

# Now process data from display GPS positions

In [6]:
# remove all duplicated long lat positions
unique_coords_df = concatenated_df.groupby('abs_loc').last().reset_index()
unique_coords_df = unique_coords_df.sort_values(by='timestamp')
unique_coords_df

,abs_loc,ACCELEROMETER X (m/s²),ACCELEROMETER Y (m/s²),ACCELEROMETER Z (m/s²),LOCATION Accuracy ( m),LOCATION Altitude ( m),LOCATION Altitude-google ( m),LOCATION Latitude :,LOCATION Longitude :,LOCATION ORIENTATION (°),LOCATION Speed ( Kmh),PROXIMITY (i),SOUND LEVEL (dB),Satellites in range,Time since start in ms,YYYY-MO-DD HH-MI-SS_SSS,timestamp,Speed(km/h),abs_acceleration
5924,51.533942,0.482000,3.415000,8.343000,1925.806400,251.6000,NaN,51.533510,-0.210930,161.5,3.500,0.00,42.95000,0 / 18,75684,2020-11-12 12:59:07:682,2020-11-12 12:59:07.682,3.500,9.027746
7174,51.536576,-1.585000,3.336000,9.987000,51.870970,251.6000,NaN,51.536160,-0.207090,161.5,3.500,0.00,47.44000,11 / 29,76693,2020-11-12 12:59:08:691,2020-11-12 12:59:08.691,3.500,10.648065
6927,51.536506,-3.371000,3.309000,8.742000,24.206450,306.5000,NaN,51.536090,-0.207140,NaN,0.000,0.00,49.21000,8 / 29,77743,2020-11-12 12:59:09:741,2020-11-12 12:59:09.741,0.000,9.936583
6888,51.536496,-5.127000,2.027000,7.891000,17.290320,316.1000,NaN,51.536080,-0.207090,196.1,1.200,0.00,52.19000,11 / 29,78759,2020-11-12 12:59:10:757,2020-11-12 12:59:10.757,1.200,9.626149
6952,51.536516,-6.543000,1.045000,8.135000,13.832260,271.0000,NaN,51.536100,-0.207160,NaN,0.000,0.00,54.70000,11 / 29,79743,2020-11-12 12:59:11:741,2020-11-12 12:59:11.741,0.000,10.491954
7002,51.536526,-5.720000,2.456000,7.803000,13.832260,287.1000,NaN,51.536110,-0.207170,39.3,1.400,0.00,55.60000,11 / 29,80733,2020-11-12 12:59:12:731,2020-11-12 12:59:12.731,1.400,9.981841
7005,51.536527,-8.272000,4.076000,5.970000,13.832260,274.2000,NaN,51.536110,-0.207230,NaN,0.000,0.00,52.47000,11 / 29,81746,2020-11-12 12:59:13:744,2020-11-12 12:59:13.744,0.000,10.985475
7081,51.536536,-2.151000,5.343000,8.537000,10.374190,251.6000,NaN,51.536120,-0.207190,60.7,3.100,0.00,51.14000,15 / 30,82763,2020-11-12 12:59:14:761,2020-11-12 12:59:14.761,3.100,10.298292
7080,51.536536,1.306000,3.486000,9.783000,10.374190,238.7000,NaN,51.536120,-0.207180,205.0,2.700,0.00,59.38000,15 / 30,83754,2020-11-12 12:59:15:752,2020-11-12 12:59:15.752,2.700,10.467326
7004,51.536527,3.045000,-2.145000,11.200000,10.374190,222.6000,NaN,51.536110,-0.207200,NaN,0.000,0.00,48.13000,15 / 30,86743,2020-11-12 12:59:18:741,2020-11-12 12:59:18.741,0.000,11.803095


In [7]:
unique_coords_df["gps_acceleration"] = unique_coords_df["LOCATION Speed ( Kmh)"].diff()
unique_coords_df["power"] = unique_coords_df["gps_acceleration"] * unique_coords_df["LOCATION Speed ( Kmh)"] * 80

#unique_coords_df.to_csv(
#    r"D:\OneDrive - Imperial College London\University Storage\Masters project\Raw data\combined.csv")

In [8]:
unique_coords_df["slope"] = unique_coords_df["LOCATION Altitude ( m)"].diff()
unique_coords_df["slope"].describe()

count    12078.000000
mean         0.001604
std          4.810844
min       -161.290300
25%          0.000000
50%          0.000000
75%          0.000000
max         87.096800
Name: slope, dtype: float64

In [9]:
unique_coords_df["slope"].value_counts().sort_index()

-161.2903     1
-154.8387     1
-122.5807     1
-112.9032     1
-93.5483      1
-74.1936      1
-67.7419      1
-58.0646      1
-48.3871      1
-45.1613      1
-45.1613      1
-45.1613      1
-45.1000      1
-41.9355      1
-41.9355      1
-38.7097      1
-35.4839      1
-32.2580      2
-29.0323      1
-25.8065      1
-25.8064      1
-22.6000      1
-22.5807      3
-22.5807      1
-22.5806      1
-22.5806      3
-19.3549      2
-19.3549      2
-19.3548      5
-19.3548      1
             ..
 9.7000       2
 12.9000      1
 12.9032     10
 12.9032      7
 12.9032      3
 12.9033      7
 16.1000      1
 16.1290      3
 16.1290      2
 16.1290      4
 16.1291      2
 16.1291      1
 19.3548      3
 19.3549      1
 19.3549      1
 22.5806      4
 25.8064      1
 25.8065      1
 25.8065      2
 29.0322      1
 29.0323      1
 32.2580      1
 32.2581      2
 35.4839      1
 41.9774      1
 51.6129      1
 54.9000      1
 61.2903      1
 74.1935      1
 87.0968      1
Name: slope, Length: 131

In [ ]:
unique_coords_df["slope"].mask(unique_coords_df["slope"].between(-200, -10), inplace=True)
unique_coords_df["slope"].mask(unique_coords_df["slope"].between(10, 200), inplace=True)

In [27]:
# Display GPS positions
fig = px.scatter_mapbox(unique_coords_df,
                        lat="LOCATION Latitude : ",
                        lon="LOCATION Longitude : ",
                        color="Speed(km/h)",#"slope",#"LOCATION Altitude ( m)",,#"Speed(km/h)", # "abs_acceleration" or "gps_acceleration" or "power"
                        zoom=14,
                        hover_data=["YYYY-MO-DD HH-MI-SS_SSS", "LOCATION Altitude ( m)"],
                        #size="LOCATION Accuracy ( m)"
                       )

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.write_html("plot.html")
fig.show()

# Filter slope values

In [ ]:
# We get a triangular window with 60 samples.
h = sg.get_window('triang', 60)
# We convolve the signal with this window.
unique_coords_df["fil"] = sg.convolve(unique_coords_df["slope"], h / h.sum())

# Display slope vs time

In [ ]:
# Display accelration
fig = px.line(unique_coords_df,
              x="YYYY-MO-DD HH-MI-SS_SSS",
              y=["LOCATION Altitude ( m)"],
              #y=["slope"],

              title='altitude'
             )
fig.write_html("plot_slope.html")
fig.show()

# Now attempt to display the plot of positions and acceleration in one page

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=1, specs=[
                        [{"type": "mapbox"}],
                        [{"type": "xy"}]
                        ])



fig.append_trace(
    go.Scattermapbox(
        lat=unique_coords_df["LOCATION Latitude : "],
        lon=unique_coords_df["LOCATION Longitude : "],
        mode='markers')
        ,row=1, col=1)



fig.append_trace(
    go.Scatter(x=concatenated_df["date_time"],
               y=concatenated_df["ACCELEROMETER X (m/s²)"])
    , row=2, col=1)


fig.append_trace(
    go.Scatter(x=concatenated_df["date_time"],
               y=concatenated_df["ACCELEROMETER Z (m/s²)"])
    , row=2, col=1)

fig.append_trace(
    go.Scatter(x=concatenated_df["date_time"],
               y=concatenated_df["ACCELEROMETER Y (m/s²)"])
    , row=2, col=1)

fig.update_layout(title_text="Stacked Subplots")
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
#mapboxt = open(".mapbox_token").read().rstrip() #my mapbox_access_token 
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Montreal 1', 'Montreal 2'),
    specs=[[{"type": "mapbox"}],
           [{"type": "mapbox"}]]
)

fig.add_trace(go.Scattermapbox(
        lat=[45.5017],
        lon=[-73.5673],
        mode='markers',
        marker_size=14,   
        text=['Montreal 1'],
    ), 1, 1)


fig.add_trace(go.Scattermapbox(
        lat=[45.6017],
        lon=[-73.7673],
        mode='markers',
        marker_size=14,
        text=['Montreal 2'],
    ), 1, 2)
fig.update_layout(
    autosize=True,
    hovermode='closest')

#update the common attributes:
fig.update_mapboxes(
        bearing=0,
        #accesstoken=mapboxt,
        center=dict(
            lat=45,
            lon=-73
        ),
        pitch=0,
        zoom=5)
   
#update different styles:
fig.update_layout(mapbox_style='open-street-map')